In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install utm

  Preparing metadata (setup.py) ... done
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6084 sha256=3650cf02aba575132fce5b931a0767b09327233cdd7fddfd38a0fff189d30fab
  Stored in directory: /root/.cache/pip/wheels/2f/a1/c8/543df0e8f5e824c3e92a432e32deb9cd89ae686095ee8cfcbe
Successfully built utm


In [ ]:
import gps2vec
import math

In [ ]:
import pandas as pd
import numpy as np
import json
import ast


In [ ]:
nrows = 20
ncols = 20
sigma = 20000
flag=1
if flag==0:
    basedir="/content/drive/MyDrive/models_visual"
elif flag==1:
    basedir="/content/drive/MyDrive/models_tag"

location=[55.755713, 37.617874]
geofea = gps2vec.georep(location,basedir,nrows,ncols,sigma,flag)
print(geofea)
print(sum(geofea))


[2.4685669e-03 2.3986280e-03 4.1378969e-03 ... 2.1014990e-04 5.0017758e-07
 5.1149696e-06]
0.9999998973495252


In [ ]:
geofea.shape

(2000,)

In [ ]:
df = pd.read_csv('data/full.csv')

In [ ]:
df

,0
0,"{'value': 'г Москва, Софийская наб, д 26/1', '..."
1,"{'value': 'г Санкт-Петербург, Лахтинский пр-кт..."
2,NaN
3,"{'value': 'г Москва, Сретенский б-р, д 11', 'u..."
4,"{'value': 'г Москва, ул Новая Басманная, д 2/1..."
...,...
53357,"{'value': 'Свердловская обл, г Каменск-Уральск..."
53358,"{'value': 'г Санкт-Петербург, ул Парадная, д 8..."
53359,"{'value': 'Краснодарский край, г Анапа, Пионер..."
53360,"{'value': 'г Новосибирск, ул Кирова, д 82', 'u..."


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from tqdm import tqdm

In [ ]:
emb = []
err = 0
for k in tqdm(range(len(df))):
  i = df.iloc[k, 0]
  if isinstance(i, float):
    emb.append(np.nan)
  else:
    try:
      d = ast.literal_eval(i)
      lat = float(d['data']['geo_lat'])
      lon = float(d['data']['geo_lon'])
      location=[lat, lon]
      geofea = gps2vec.georep(location,basedir,nrows,ncols,sigma,flag)
      emb.append(geofea)
    except:
      emb.append(np.nan)
      err +=1


100%|██████████| 53362/53362 [1:22:09<00:00, 10.83it/s]


In [ ]:
emb_df = pd.Series(emb)

In [ ]:
emb_df

0        [0.0021982712, 0.0025602274, 0.004451619, 0.00...
1        [0.005155264, 0.0012874127, 0.0076143104, 0.00...
2                                                      NaN
3        [0.0027381002, 0.0022334736, 0.004270247, 0.00...
4        [0.0025980012, 0.0022231229, 0.004554158, 0.00...
                               ...                        
53357                                                  NaN
53358    [0.004588805, 0.0015600106, 0.0018179208, 0.00...
53359                                                  NaN
53360                                                  NaN
53361                                                  NaN
Length: 53362, dtype: object

In [ ]:
emb_np = emb_df.to_numpy()

In [ ]:
emb_np.save()

In [ ]:
np.save('/content/drive/MyDrive/emb_np.npy', emb_np)

In [ ]:
a = np.load('/content/drive/MyDrive/emb_np.npy', allow_pickle=True)

In [ ]:
len(a)

53362

In [ ]:
len(a[0])

2000

In [ ]:
len(emb_np[0])

2000

In [ ]:
emb_df.to_excel('/content/drive/MyDrive/emb_df.xlsx')

In [ ]:
df1 = pd.read_excel('/content/drive/MyDrive/emb_df.xlsx')
df1

,Unnamed: 0,0
0,0,[2.1982712e-03 2.5602274e-03 4.4516190e-03 ......
1,1,[5.1552639e-03 1.2874127e-03 7.6143104e-03 ......
2,2,NaN
3,3,[2.7381002e-03 2.2334736e-03 4.2702472e-03 ......
4,4,[2.5980012e-03 2.2231229e-03 4.5541581e-03 ......
...,...,...
53357,53357,NaN
53358,53358,[4.5888051e-03 1.5600106e-03 1.8179208e-03 ......
53359,53359,NaN
53360,53360,NaN


In [ ]:
df1.iloc[0, 1]

'[2.1982712e-03 2.5602274e-03 4.4516190e-03 ... 2.2083297e-04 4.3148430e-07\n 2.6487130e-06]'

In [ ]:
math.nan == df.iloc[2,0]

False

In [ ]:
math.isnan(df.iloc[2,0])

True

In [ ]:
df.iloc[0,0]

"{'value': 'г Москва, Софийская наб, д 26/1', 'unrestricted_value': '115035, г Москва, р-н Якиманка, Софийская наб, д 26/1', 'data': {'postal_code': '115035', 'country': 'Россия', 'country_iso_code': 'RU', 'federal_district': 'Центральный', 'region_fias_id': '0c5b2444-70a0-4932-980c-b4dc0d3f02b5', 'region_kladr_id': '7700000000000', 'region_iso_code': 'RU-MOW', 'region_with_type': 'г Москва', 'region_type': 'г', 'region_type_full': 'город', 'region': 'Москва', 'area_fias_id': None, 'area_kladr_id': None, 'area_with_type': None, 'area_type': None, 'area_type_full': None, 'area': None, 'city_fias_id': '0c5b2444-70a0-4932-980c-b4dc0d3f02b5', 'city_kladr_id': '7700000000000', 'city_with_type': 'г Москва', 'city_type': 'г', 'city_type_full': 'город', 'city': 'Москва', 'city_area': 'Центральный', 'city_district_fias_id': None, 'city_district_kladr_id': None, 'city_district_with_type': 'р-н Якиманка', 'city_district_type': 'р-н', 'city_district_type_full': 'район', 'city_district': 'Якиманка'

(55.746958, 37.620151)

In [ ]:
math.isnan('qwer')

TypeError: must be real number, not str

In [ ]:
sup = np.load('sup_cords.npy', allow_pickle=True)

In [ ]:
sup

array([list([55.682955, 37.848655]), nan, nan, ...,
       list([55.75396, 37.620393]), list([59.896963, 30.2858368]),
       list([55.044433, 82.884928])], dtype=object)

In [ ]:
emb_sup = []
err = 0
mama = 0
for k in tqdm(range(len(sup))):
  i = sup[k]
  if isinstance(i, float):
    emb_sup.append(np.nan)
    mama+=1
  else:
    try:
      lat = i[0]
      lon = i[1]
      location=[lat, lon]
      geofea = gps2vec.georep(location,basedir,nrows,ncols,sigma,flag)
      emb_sup.append(geofea)
    except:
      emb_sup.append(np.nan)
      err +=1


100%|██████████| 3134/3134 [06:05<00:00,  8.57it/s]


In [ ]:
emb_sup

[array([1.8463489e-03, 2.7800016e-03, 7.8708585e-03, ..., 1.7175405e-06,
        4.3210683e-08, 2.5274622e-09], dtype=float32),
 nan,
 nan,
 array([2.9145407e-03, 1.0541924e-03, 1.3512766e-03, ..., 3.1318385e-05,
        2.0480709e-07, 1.2479371e-03], dtype=float32),
 nan,
 nan,
 array([3.6840818e-03, 1.3062886e-03, 1.8439225e-03, ..., 4.4846343e-05,
        3.0477935e-07, 1.1632285e-03], dtype=float32),
 array([1.3260222e-03, 1.0152762e-03, 2.4067110e-03, ..., 1.2982365e-06,
        6.8249690e-08, 1.2439315e-07], dtype=float32),
 nan,
 nan,
 nan,
 nan,
 nan,
 array([3.7876458e-03, 3.3160059e-03, 4.6729129e-03, ..., 6.6146282e-07,
        9.7367128e-08, 1.7350337e-03], dtype=float32),
 nan,
 nan,
 nan,
 nan,
 array([2.6483850e-03, 7.6654507e-04, 2.6474494e-04, ..., 9.9906117e-07,
        1.2207116e-07, 8.8703656e-04], dtype=float32),
 array([5.4334518e-03, 2.4609263e-03, 4.3244050e-03, ..., 1.4050762e-06,
        3.8191581e-07, 2.9678236e-05], dtype=float32),
 nan,
 array([3.9086468e-0

In [ ]:
emb_df = pd.Series(emb_sup)

In [ ]:
emb_df

0       [0.0018463489, 0.0027800016, 0.007870859, 0.00...
1                                                     NaN
2                                                     NaN
3       [0.0029145407, 0.0010541924, 0.0013512766, 0.0...
4                                                     NaN
                              ...                        
3129                                                  NaN
3130    [0.002403069, 0.0024190692, 0.0042488533, 0.00...
3131    [0.002403069, 0.0024190692, 0.0042488533, 0.00...
3132    [0.004482255, 0.0014311897, 0.001494439, 0.006...
3133                                                  NaN
Length: 3134, dtype: object

In [ ]:
emb_np = emb_df.to_numpy()

In [ ]:
np.save('/content/drive/MyDrive/emb_np_sup.npy', emb_np)

In [ ]:
a = np.load('/content/drive/MyDrive/emb_np_sup.npy', allow_pickle=True)

In [ ]:
len(a)

3134